In [25]:
%%bash
which python3
which pip
which jupyter
pip freeze --user

/hb/home/swoonna/anaconda3/envs/sbp/bin/python3
/hb/home/swoonna/anaconda3/envs/sbp/bin/pip
/hb/home/swoonna/anaconda3/envs/sbp/bin/jupyter
absl-py==0.6.1
alembic==1.0.5
astor==0.7.1
bsddb3==6.2.6
bz2file==0.98
cachetools==3.0.0
chardet==3.0.4
Click==7.0
Flask==1.0.2
future==0.17.1
gast==0.2.1.post0
gevent==1.3.7
google-api-python-client==1.7.6
google-auth==1.6.2
google-auth-httplib2==0.0.3
googleapis-common-protos==1.5.5
greenlet==0.4.15
grpcio==1.17.1
gunicorn==19.9.0
Gutenberg==0.7.0
gym==0.10.9
httplib2==0.12.0
idna==2.8
isodate==0.6.0
itsdangerous==1.1.0
Jinja2==2.10
Keras-Applications==1.0.6
Keras-Preprocessing==1.0.5
Mako==1.0.7
Markdown==3.0.1
MarkupSafe==1.1.0
mesh-tensorflow==0.0.5
mpmath==1.1.0
nltk==3.4
numpy==1.15.4
oauth2client==4.1.3
pandas==0.23.4
promise==2.2.1
protobuf==3.6.1
pyasn1==0.4.5
pyasn1-modules==0.2.2
pyglet==1.3.2
python-editor==1.0.3
pytz==2018.9
rdflib==4.2.2
rdflib-sqlalchemy==0.3.8
requests==2.21.0
rsa==4.0
scipy==1.2.0
singledispatch==3.4.0.3
SQLAlchem

In [26]:
%%bash
pip install tensorflow==1.12 --user
pip install tensor2tensor --user

In [27]:
%%bash
pip freeze --user
#python -m pip freeze --user

absl-py==0.6.1
alembic==1.0.5
astor==0.7.1
bsddb3==6.2.6
bz2file==0.98
cachetools==3.0.0
chardet==3.0.4
Click==7.0
Flask==1.0.2
future==0.17.1
gast==0.2.1.post0
gevent==1.3.7
google-api-python-client==1.7.6
google-auth==1.6.2
google-auth-httplib2==0.0.3
googleapis-common-protos==1.5.5
greenlet==0.4.15
grpcio==1.17.1
gunicorn==19.9.0
Gutenberg==0.7.0
gym==0.10.9
httplib2==0.12.0
idna==2.8
isodate==0.6.0
itsdangerous==1.1.0
Jinja2==2.10
Keras-Applications==1.0.6
Keras-Preprocessing==1.0.5
Mako==1.0.7
Markdown==3.0.1
MarkupSafe==1.1.0
mesh-tensorflow==0.0.5
mpmath==1.1.0
nltk==3.4
numpy==1.15.4
oauth2client==4.1.3
pandas==0.23.4
promise==2.2.1
protobuf==3.6.1
pyasn1==0.4.5
pyasn1-modules==0.2.2
pyglet==1.3.2
python-editor==1.0.3
pytz==2018.9
rdflib==4.2.2
rdflib-sqlalchemy==0.3.8
requests==2.21.0
rsa==4.0
scipy==1.2.0
singledispatch==3.4.0.3
SQLAlchemy==1.2.15
sympy==1.3
tensor2tensor==1.11.0
tensorboard==1.12.2
tensorflow==1.12.0
tensorflow-gpu==1.12.0
tensorflow-metadata==0.9.0
tensorfl

In [28]:
import pkg_resources
#import csv
pkg_resources.require("Tensorflow==1.12.0")
import tensorflow as tf
tf.__version__

'1.12.0'

In [29]:
import os
# this is what this notebook is demonstrating
PROBLEM= 'lang_gen_problem'
os.environ['PROBLEM'] = PROBLEM

## Training

In [33]:
%%bash
rm -rf lang_model
mkdir -p lang_model/trainer

In [34]:
%%writefile lang_model/trainer/problem.py
import os
import tensorflow as tf
from tensor2tensor.utils import registry
from tensor2tensor.models import transformer
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import generator_utils

@registry.register_problem
class LangGenProblem(text_problems.Text2SelfProblem):

  @property
  def approx_vocab_size(self):
    return 40000  # ~8k

  @property
  def is_generate_per_split(self):
    # generate_data will NOT shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 10% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 90,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 10,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    with open('data/lang_gen/raw.txt', 'r') as rawfp:
        for curr_line in rawfp:
            curr_line = curr_line.strip()
            if len(curr_line) > 0:       
                yield {
                    "targets": curr_line
                }        


# Smaller than the typical translate model, and with more regularization
@registry.register_hparams
def transformer_lang_gen():
  hparams = transformer.transformer_base()
  hparams.num_hidden_layers = 2
  hparams.hidden_size = 128
  hparams.filter_size = 512
  hparams.num_heads = 4
  hparams.attention_dropout = 0.6
  hparams.layer_prepostprocess_dropout = 0.6
  hparams.learning_rate = 0.05
  return hparams


Writing lang_model/trainer/problem.py


In [35]:
%%writefile lang_model/trainer/__init__.py
from . import problem

Writing lang_model/trainer/__init__.py


In [36]:
!head data/lang_gen/*.txt

../data/ParlAI/setup.py
#!/usr/bin/env python3

# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree. An additional grant
# of patent rights can be found in the PATENTS file in the same directory.




In [37]:
%%writefile lang_model/setup.py
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
  'tensor2tensor'
]

setup(
    name='lang_model',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description="Lang Gen Problem",
    requires=[]
)

Writing lang_model/setup.py


In [38]:
!touch lang_model/__init__.py

In [39]:
!find lang_model/

lang_model/
lang_model/setup.py
lang_model/__init__.py
lang_model/trainer
lang_model/trainer/problem.py
lang_model/trainer/__init__.py


In [40]:
%%bash
DATA_DIR=./t2t_data
TMP_DIR=$DATA_DIR/tmp
rm -rf $DATA_DIR $TMP_DIR
mkdir -p $DATA_DIR $TMP_DIR
# Generate data
t2t-datagen \
  --t2t_usr_dir=./lang_model/trainer \
  --problem=$PROBLEM \
  --data_dir=$DATA_DIR \
  --tmp_dir=$TMP_DIR

:::MLPv0.5.0 transformer 1549925273.531204700 (/hb/home/swoonna/.local/lib/python3.6/site-packages/tensor2tensor/data_generators/text_problems.py:311) preproc_tokenize_training
:::MLPv0.5.0 transformer 1549925326.948022366 (/hb/home/swoonna/.local/lib/python3.6/site-packages/tensor2tensor/data_generators/text_problems.py:311) preproc_num_train_examples: 275356


/hb/home/swoonna/.local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/hb/home/swoonna/.local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/hb/home/swoonna/.local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/hb/home/swoonna/.local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/hb/home/swoonna/.local/lib/python3.6/site-packa

In [41]:
!ls t2t_data

lang_gen_problem-dev-00000-of-00010    lang_gen_problem-train-00041-of-00090
lang_gen_problem-dev-00001-of-00010    lang_gen_problem-train-00042-of-00090
lang_gen_problem-dev-00002-of-00010    lang_gen_problem-train-00043-of-00090
lang_gen_problem-dev-00003-of-00010    lang_gen_problem-train-00044-of-00090
lang_gen_problem-dev-00004-of-00010    lang_gen_problem-train-00045-of-00090
lang_gen_problem-dev-00005-of-00010    lang_gen_problem-train-00046-of-00090
lang_gen_problem-dev-00006-of-00010    lang_gen_problem-train-00047-of-00090
lang_gen_problem-dev-00007-of-00010    lang_gen_problem-train-00048-of-00090
lang_gen_problem-dev-00008-of-00010    lang_gen_problem-train-00049-of-00090
lang_gen_problem-dev-00009-of-00010    lang_gen_problem-train-00050-of-00090
lang_gen_problem-train-00000-of-00090  lang_gen_problem-train-00051-of-00090
lang_gen_problem-train-00001-of-00090  lang_gen_problem-train-00052-of-00090
lang_gen_problem-train-00002-of-00090  lang_gen_problem-train-00053-of-00090

In [42]:
%%bash
rm -r lang_model/data
mkdir lang_model/data
DATA_DIR=./t2t_data
cp ${DATA_DIR}/${PROBLEM}* ${DATA_DIR}/vocab* lang_model/data

rm: cannot remove ‘lang_model/data’: No such file or directory


In [ ]:
%%bash
mkdir lang_model/subset
BASE=lang_model/data
OUTDIR=lang_model/subset
cp \
    ${BASE}/${PROBLEM}-train-0008* \
    ${BASE}/${PROBLEM}-dev-00000*  \
    ${BASE}/vocab* \
    $OUTDIR

In [43]:
%%writefile encoder.sh
#!/bin/bash
DATA_DIR=lang_model/data
OUTDIR=./trained_model
rm -rf $OUTDIR
t2t-trainer \
  --data_dir=lang_model/data \
  --t2t_usr_dir=./lang_model/trainer \
  --problem=$PROBLEM \
  --model=transformer \
  --hparams_set=transformer_lang_gen \
  --output_dir=$OUTDIR --job-dir=$OUTDIR --train_steps=200

Overwriting encoder.sh


In [44]:
%%writefile sbp.slurm
#!/bin/bash
#note - there can be no empty lines between #SBATCH directives.
#SBATCH --job-name=$swoonna-gpu
#SBATCH --output=output/encoder.out
#SBATCH --error=output/encoder.err
#SBATCH --nodes=1
#SBATCH --mem=80GB
#SBATCH --cpus-per-task=10    # CPU cores/threads
#SBATCH --mail-type=END,FAIL
#SBATCH --mail-user=sanjana_woona@yahoo.co.in
#SBATCH --partition=96x24gpu4
# The following designates you are using GPU 1 - Tesla P100 model
# Please only use 1GPU for 1 job at a time.
#SBATCH --gres=gpu:p100:1

chmod +x encoder.sh
./encoder.sh

Overwriting sbp.slurm


In [45]:
!sbatch sbp.slurm 

Submitted batch job 65120


In [47]:
!squeue -u swoonna

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             65120 96x24gpu4 $swoonna  swoonna  R       0:08      1 hbcgpu-021


In [ ]:
!scancel

## Generate samples

In [19]:
%%writefile input.txt



Overwriting input.txt


In [20]:
%%writefile sampling.sh
#!/bin/bash
# same as the above training job ...
OUTDIR=./trained_model 
DATADIR=./lang_model/data
MODEL=transformer

BEAM_SIZE=5
ALPHA=0.6

t2t-decoder \
  --data_dir=$DATADIR \
  --problem=$PROBLEM \
  --model=$MODEL \
  --hparams_set=transformer_lang_gen \
  --output_dir=$OUTDIR \
  --t2t_usr_dir=./lang_model/trainer \
  --decode_hparams="beam_size=$BEAM_SIZE,alpha=$ALPHA" \
  --hparams='sampling_method=random' \
  --decode_from_file=input.txt \ 

Overwriting sampling.sh


In [21]:
%%writefile code_gen.slurm
#!/bin/bash
#note - there can be no empty lines between #SBATCH directives.
#SBATCH --job-name=$swoonna-gpu
#SBATCH --output=output/decoder.out
#SBATCH --error=output/decoder.err
#SBATCH --nodes=1
# allocate 5GB or RAM on node. You must declare --mem in all scripts
#SBATCH --mem=5GB
#SBATCH --cpus-per-task=4
#SBATCH --mail-type=END,FAIL
#SBATCH --mail-user=sanjana_woona@yahoo.co.in
#SBATCH --partition=96x24gpu4
# The following designates you are using GPU 1 - Tesla P100 model
# Please only use 1GPU for 1 job at a time.
#SBATCH --gres=gpu:p100:1

chmod +x sampling.sh
./sampling.sh

Overwriting code_gen.slurm


In [48]:
!sbatch code_gen.slurm 

Submitted batch job 65121


In [50]:
!squeue -u swoonna

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             65120 96x24gpu4 $swoonna  swoonna  R       3:28      1 hbcgpu-021
             65121 96x24gpu4 $swoonna  swoonna  R       0:23      1 hbcgpu-021


In [13]:
!scancel 